# Issue [#305]
## In Python LangGraph, there is a feature called Private State that allows internal nodes to pass information that is not required in the graph's input/output.

Try to resolve issue using LangGraph4j Hooks

[#305]: https://github.com/langgraph4j/langgraph4j/issues/305

In [1]:
var userHomeDir = System.getProperty("user.home");
var localRespoUrl = "file://" + userHomeDir + "/.m2/repository/";
var langgraph4jVersion = "1.8-SNAPSHOT";

In [2]:
%%bash 
rm -rf \{userHomeDir}/Library/Jupyter/kernels/rapaio-jupyter-kernel/mima_cache/org/bsc/langgraph4j

In [3]:
%dependency /add-repo local \{localRespoUrl} release|never snapshot|always
%dependency /add org.slf4j:slf4j-jdk14:2.0.9
%dependency /add org.bsc.langgraph4j:langgraph4j-core:\{langgraph4jVersion}

%dependency /resolve

Repository local url: file:///Users/bsorrentino/.m2/repository/ added.
Adding dependency org.slf4j:slf4j-jdk14:2.0.9
Adding dependency org.bsc.langgraph4j:langgraph4j-core:1.8-SNAPSHOT
Solving dependencies
Resolved artifacts count: 4
Add to classpath: /Users/bsorrentino/Library/Jupyter/kernels/rapaio-jupyter-kernel/mima_cache/org/slf4j/slf4j-jdk14/2.0.9/slf4j-jdk14-2.0.9.jar
Add to classpath: /Users/bsorrentino/Library/Jupyter/kernels/rapaio-jupyter-kernel/mima_cache/org/slf4j/slf4j-api/2.0.9/slf4j-api-2.0.9.jar
Add to classpath: /Users/bsorrentino/Library/Jupyter/kernels/rapaio-jupyter-kernel/mima_cache/org/bsc/langgraph4j/langgraph4j-core/1.8-SNAPSHOT/langgraph4j-core-1.8-SNAPSHOT.jar
Add to classpath: /Users/bsorrentino/Library/Jupyter/kernels/rapaio-jupyter-kernel/mima_cache/org/bsc/async/async-generator/4.0.0/async-generator-4.0.0.jar


### Imports

In [11]:
import org.bsc.langgraph4j.*;
import org.slf4j.*;


import org.bsc.langgraph4j.action.AsyncNodeActionWithConfig;
import org.bsc.langgraph4j.hook.NodeHook;
import org.bsc.langgraph4j.prebuilt.MessagesState;
import org.bsc.langgraph4j.state.AgentState;
import org.bsc.langgraph4j.state.Channel;
import org.bsc.langgraph4j.utils.CollectionsUtils;

import java.util.*;
import java.util.concurrent.*;
import java.util.stream.*;
import java.util.function.*;

import static java.util.concurrent.CompletableFuture.completedFuture;
import static org.bsc.langgraph4j.utils.CollectionsUtils.mergeMap;
import static org.bsc.langgraph4j.GraphDefinition.END;
import static org.bsc.langgraph4j.GraphDefinition.START;
import static org.bsc.langgraph4j.action.AsyncNodeActionWithConfig.node_async;
import static java.util.Objects.requireNonNull;

try( var file = new java.io.FileInputStream("./logging.properties")) { // INITIALIZE LOGGING
    java.util.logging.LogManager.getLogManager().readConfiguration( file );
}


## Hooks for logging

In [12]:


public record LoggingNodeHook<State extends AgentState>()
        implements NodeHook.WrapCall<State>, LG4JLoggable {

    @Override
    public CompletableFuture<Map<String, Object>> applyWrap( String nodeId, State state, RunnableConfig config, AsyncNodeActionWithConfig<State> action) {

        log.info("node action fo node '{}' start with state: {}", nodeId, state);
        return action.apply(state, config)
                .whenComplete( ( result, exception ) -> {
                    log.info("node action fo node '{}' end with request update: {}", nodeId, CollectionsUtils.toString(result));
                });
    }
}


In [13]:
static class State extends MessagesState<String> {

    public State(Map<String, Object> initData) {
        super(initData);
    }

}

## Apply Logging Hook

In [14]:
AsyncNodeActionWithConfig<State> simpleAction() {
    return node_async( ( state, config ) -> Map.of( "messages", config.nodeId() ) );
}

AsyncNodeActionWithConfig<State> actionProducePrivateStateAttribute() {
    return node_async( ( state, config ) -> 
        Map.of( "messages", config.nodeId(), "private_key", "private_value" ) );
}

AsyncNodeActionWithConfig<State> actionConsumePrivateStateAttribute() {
    return node_async( ( state, config ) -> {

        var privateValue = state.value("private_key").orElseThrow();
        LG4JLoggable.log.info("Got Private value: {}", privateValue);

        return Map.of( "messages", config.nodeId() );

    });
}

var stateGraph = new StateGraph<>(MessagesState.SCHEMA, State::new)
        .addWrapCallNodeHook( new LoggingNodeHook<>() )
        .addNode("node_1", simpleAction() )
        .addNode("node_2", actionProducePrivateStateAttribute() )
        .addNode("node_3", actionConsumePrivateStateAttribute() )
        .addNode("node_4", simpleAction() )
        .addEdge(START, "node_1")
        .addEdge("node_1", "node_2")
        .addEdge("node_2", "node_3")
        .addEdge("node_3", "node_4")
        .addEdge("node_4", END);




In [15]:
var workflow = stateGraph.compile();

var result = workflow.invoke( GraphInput.noArgs(), RunnableConfig.builder().build());
    
LG4JLoggable.log.info( "Workflow execution result:{}", result.orElseThrow() );

START 
node action fo node 'node_1' start with state: {
	messages=[]
} 
node action fo node 'node_1' end with request update: {
	messages=node_1
} 
node action fo node 'node_2' start with state: {
	messages=[
	node_1
	]
} 
node action fo node 'node_2' end with request update: {
	messages=node_2
	private_key=private_value
} 
node action fo node 'node_3' start with state: {
	messages=[
	node_1
	node_2
	]
	private_key=private_value
} 
Got Private value: private_value 
node action fo node 'node_3' end with request update: {
	messages=node_3
} 
node action fo node 'node_4' start with state: {
	messages=[
	node_1
	node_2
	node_3
	]
	private_key=private_value
} 
node action fo node 'node_4' end with request update: {
	messages=node_4
} 
Workflow execution result:{
	messages=[
	node_1
	node_2
	node_3
	node_4
	]
	private_key=private_value
} 


## Add an Hook to remove private results 

The following Node Hook is invoked after node call that allow to enrich the original result  

In [18]:
import java.util.stream.Collectors;

public <State extends AgentState> NodeHook.AfterCall<State> removeAttributesHook( List<String> attributeKeysToRemove ) {
    return ( nodeId, state, config, lastResult ) -> {
        
        final var attributeKeysToRemoveMap = attributeKeysToRemove.stream()
                .collect( Collectors.toMap( key -> key, key -> AgentState.MARK_FOR_REMOVAL ) );

        return completedFuture( mergeMap( lastResult, attributeKeysToRemoveMap, (l, r) -> r ) );
        
    };
}



## Apply Hook to remove private keys after evaluated node: "node_3"

In [19]:

var workflow = stateGraph
                .addAfterCallNodeHook( "node_3", removeAttributesHook( List.of("private_key") ) )      
                .compile();

var result = workflow.invoke( GraphInput.noArgs(), RunnableConfig.builder().build());
    
LG4JLoggable.log.info( "Workflow execution result:{}", result.orElseThrow() );

START 
node action fo node 'node_1' start with state: {
	messages=[]
} 
node action fo node 'node_1' end with request update: {
	messages=node_1
} 
node action fo node 'node_2' start with state: {
	messages=[
	node_1
	]
} 
node action fo node 'node_2' end with request update: {
	messages=node_2
	private_key=private_value
} 
node action fo node 'node_3' start with state: {
	messages=[
	node_1
	node_2
	]
	private_key=private_value
} 
Got Private value: private_value 
node action fo node 'node_3' end with request update: {
	messages=node_3
} 
node action fo node 'node_4' start with state: {
	messages=[
	node_1
	node_2
	node_3
	]
} 
node action fo node 'node_4' end with request update: {
	messages=node_4
} 
Workflow execution result:{
	messages=[
	node_1
	node_2
	node_3
	node_4
	]
} 
